In [21]:
import random
import pandas as pd

In [22]:
POPULATION_SIZE = 15
TAB_SIZE = 8

Função pra geração do FitnessScore

In [23]:
'''
A função definida para encontrar o FitnessScore, corre por todas as colunas do tabuleiro,
verificando se a rainha dessa coluna se encontra na mesma linha que alguma rainha anterior
e verificando também se há outras rainhas nas mesmas diagonais.
A cada par de colisão encontrado, o fitness score é aumentado em 1.

aux1 = posição da rainha da coluna atual em relação ao eixo y
aux2 = posição da rainha à qual 'aux1' vai ser comparada

i = coluna da rainha atual
j = coluna da rainha a ser comparada
'''
def getFitnessScore(solution):
    fitness_score = 0
    for i in range(0, TAB_SIZE):
        for j in range(0, i):
            aux1 = solution[i]
            aux2 = solution[j]

            if (aux1 == aux2) or (j - i == aux2 - aux1) or (j - i == aux1 - aux2):
                fitness_score += 1
    return fitness_score

Gerando a População Inicial

In [24]:
'''
Para cada iteração da população, é gerado um vetor vazio, que tem seus valores sorteados para
representação da solução gerada. Após a geração da solução, é gerado seu FitnessScore.
'''
initial_population = {'Population':[]}

for i in range(0, POPULATION_SIZE):
    random_list = []
    fitness_score = 0
    for j in range(0,TAB_SIZE):            
        random_list.append(random.randint(0,TAB_SIZE-1))    
    initial_population['Population'].append({'Solution':random_list, 'FitnessScore':getFitnessScore(random_list)})

In [25]:
print(initial_population)

{'Population': [{'Solution': [7, 6, 0, 3, 2, 1, 6, 3], 'FitnessScore': 9}, {'Solution': [1, 7, 1, 3, 7, 3, 6, 3], 'FitnessScore': 7}, {'Solution': [5, 5, 3, 1, 0, 4, 1, 4], 'FitnessScore': 5}, {'Solution': [6, 2, 1, 3, 0, 0, 4, 4], 'FitnessScore': 4}, {'Solution': [1, 6, 7, 6, 3, 2, 3, 1], 'FitnessScore': 11}, {'Solution': [3, 0, 2, 5, 7, 1, 0, 6], 'FitnessScore': 4}, {'Solution': [0, 3, 3, 1, 1, 2, 2, 0], 'FitnessScore': 8}, {'Solution': [4, 5, 5, 4, 6, 5, 3, 0], 'FitnessScore': 9}, {'Solution': [1, 1, 7, 1, 2, 2, 6, 5], 'FitnessScore': 9}, {'Solution': [5, 0, 2, 3, 0, 2, 4, 4], 'FitnessScore': 6}, {'Solution': [2, 4, 6, 5, 2, 1, 7, 5], 'FitnessScore': 6}, {'Solution': [1, 1, 4, 7, 2, 7, 0, 1], 'FitnessScore': 9}, {'Solution': [7, 2, 3, 6, 4, 4, 5, 6], 'FitnessScore': 7}, {'Solution': [3, 6, 6, 0, 4, 5, 1, 4], 'FitnessScore': 7}, {'Solution': [6, 7, 6, 7, 1, 7, 4, 0], 'FitnessScore': 8}]}


Conversão da população inicial para um DataFrame

In [26]:
'''
Após a geração de toda a população inicial com seus respectivos FitnessScores, convertemos os dados atuais
para o formato de dataframe, para melhor visualização e manipulação dos dados.
O DataFrame é ordenado de acordo com o FitnessScore de cada solução, em ordem crescente. Dessa forma, temos
as soluções, da melhor à pior, de forma ordenada.
'''
df = pd.DataFrame(initial_population['Population']).sort_values(by=['FitnessScore']).reset_index(drop=True)
print(df)

                    Solution  FitnessScore
0   [6, 2, 1, 3, 0, 0, 4, 4]             4
1   [3, 0, 2, 5, 7, 1, 0, 6]             4
2   [5, 5, 3, 1, 0, 4, 1, 4]             5
3   [5, 0, 2, 3, 0, 2, 4, 4]             6
4   [2, 4, 6, 5, 2, 1, 7, 5]             6
5   [1, 7, 1, 3, 7, 3, 6, 3]             7
6   [7, 2, 3, 6, 4, 4, 5, 6]             7
7   [3, 6, 6, 0, 4, 5, 1, 4]             7
8   [0, 3, 3, 1, 1, 2, 2, 0]             8
9   [6, 7, 6, 7, 1, 7, 4, 0]             8
10  [7, 6, 0, 3, 2, 1, 6, 3]             9
11  [4, 5, 5, 4, 6, 5, 3, 0]             9
12  [1, 1, 7, 1, 2, 2, 6, 5]             9
13  [1, 1, 4, 7, 2, 7, 0, 1]             9
14  [1, 6, 7, 6, 3, 2, 3, 1]            11


Função que gera os filhos por cruzamento e mutação

In [27]:
'''
A função definida a seguir, usa como input o dataframe da população, e usa as duas melhores soluções atuais
para gerar seus filhos. Eu optei por gerar dois filhos com diferentes pontos de cruzamento (sorteados) apenas
para brincar com a complexidade do problema. Caso seja desejado que ambos tenham cruzamento no mesmo ponto,
é apenas necessário que a segunda linha que sorteia a variável 'loc' seja comentada.
A partir do sorteio do ponto de cruzamento, geramos dois filhos: um com a primeira parte correspondente à
primeira solução do dataframe e a segunda parte correspondente à segunda solução, e o segundo filho com as
correspondências contrárias.
Com esses dois filhos gerados, sorteamos um ponto de mutação para cada um desses filhos, em que essa posição
é substituída por um valor aleatório.
Temos então quatro filhos gerados, que são adicionados ao dataframe, que já retorna de forma ordenada.
'''
def crossover_and_mutate(df):
    sol1 = df.loc[0]['Solution']
    sol2 = df.loc[1]['Solution']

##################### CROSSOVER ###############################
    loc = random.randint(1,TAB_SIZE-1)
    new_sol1 = sol1[0:loc] + sol2[loc:]

    loc = random.randint(1,TAB_SIZE-1)
    new_sol2 = sol2[0:loc] + sol1[loc:]
    
##################### MUTATION ###############################
    loc = random.randint(0,TAB_SIZE-1)
    new_sol1_m = new_sol1
    new_sol1_m[loc] = random.randint(0,TAB_SIZE-1)

    loc = random.randint(0,TAB_SIZE-1)
    new_sol2_m = new_sol2
    new_sol2_m[loc] = random.randint(0,TAB_SIZE-1)

    df = pd.concat([df, pd.DataFrame([{'Solution':new_sol1, 'FitnessScore':getFitnessScore(new_sol1)},
                                    {'Solution':new_sol2, 'FitnessScore':getFitnessScore(new_sol2)},
                                    {'Solution':new_sol1_m, 'FitnessScore':getFitnessScore(new_sol1_m)},
                                    {'Solution':new_sol2_m, 'FitnessScore':getFitnessScore(new_sol2_m)}])], ignore_index=True)
    
    df = df.sort_values(by=['FitnessScore']).reset_index(drop=True)
    
    return df

Busca pela solução de FitnessScore desejado

In [28]:
'''
A função anterior é repetida 'n' vezes com a nova população gerada, até que o cruzamento ou mutação
das melhores soluções resulte em uma solução de FitnessScore = 0.
'''
found = (df.loc[0]['FitnessScore'] == 0)
iter = 0

while(not found):
    df = crossover_and_mutate(df)
    iter +=1
    found = (df.loc[0]['FitnessScore'] == 0)

print(f"Solução encontrada com {iter} iterações.")
print(df.loc[0])

Solução encontrada com 29706 iterações.
Solution        [2, 6, 1, 7, 5, 3, 0, 4]
FitnessScore                           0
Name: 0, dtype: object
